# Extracting Positive and negative routine recommendations

In [25]:
import pandas as pd
import numpy as np


from sklearn.preprocessing import StandardScaler, LabelBinarizer
import tensorflow_hub as hub
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
import pickle

In [26]:
# === Step 1: Import encoded dataset ===
d5f = pd.read_csv('haircare_encoded_final2.csv')
d5f2 = pd.read_csv('hair Care Survey_v2 (Responses) - Form responses 1.csv')



In [27]:
d5f2.head()

,Timestamp,Score,Age,Race,Gender,Country,Hair type,Hair porosity,Hair texture,Hair density,...,Main factor influencing your hair health or growth,Tips or products have worked well for your hair,Hair state and their cause: [Hydrated & Healthy],Hair state and their cause: [Promote Frizzy],Hair state and their cause: [Tangled],Hair state and their cause: [dryness & breaking],Would you participate in follow-up studies?,Email,Email address,SurveyCircle \n 'Thank-you-for-your-participation' page. Please use the following text line that contains your Survey Code:
0,16/04/2025 19:31,0 / 7,25-34,Black/African,Female,Namibia,4c,Low,Medium,Low,...,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN
1,23/04/2025 13:31,0 / 7,35-44,Black/African,Female,Namibia,3c,Medium,Medium,Medium,...,moisturising,gentle,Hot oil treatment,Shampoo,Shampoo,Shampoo,Yes,poppiculrs@gmail.com,NaN,NaN
2,24/04/2025 14:59,0 / 7,25-34,Black/African,Female,Namibia,3c,Medium,Medium,Medium,...,Moisturizer,NaN,Hot oil treatment,Shampoo,Shampoo,Shampoo,Yes,psndalipo@gmail.com,NaN,NaN
3,24/04/2025 16:20,0 / 7,25-34,Black/African,Female,NaN,3c,High,Coarse,High,...,Lack of consistency,NaN,Herbal oil,Hair Gel,Shampoo,Shampoo,Yes,NaN,NaN,NaN
4,28/04/2025 13:43,0 / 7,25-34,Black/African,Female,Namibia,4b,Medium,Coarse,Low,...,Lifestyle,Castor Oil and changing the hair I braid with....,Hair Masks,Shampoo,Leave-In Conditioner,Hair Gel,Yes,shyner771@gmail.com,shyner771@gmail.com,NaN


In [28]:
# Clean column names: strip whitespace, convert to lowercase, replace spaces and special chars

# Drop irrelevant columns (e.g., email, survey codes, etc.)
columns_to_drop = [
    'Timestamp', 'Score', 'Email', 'Email address',
    "SurveyCircle  \n  'Thank-you-for-your-participation' page. Please use the following text line that contains your Survey Code:",
    'Column 53', 'Would you participate in follow-up studies?'
]


d5f2.drop(columns=columns_to_drop, inplace=True, errors='ignore')

#Normalize Text Columns: Strip whitespace, Lowercase, Remove special character
d5f2.columns = (
    d5f2.columns
    .str.strip()
    .str.lower()
    .str.replace(' ', '_')
    .str.replace('[^a-zA-Z0-9_]', '', regex=True)
)

## data encoding

In [30]:

# Example dataset
data_por = d5f2["hair_porosity"]
data_brk = d5f2["hair_breakage"]

# Encode
le_porosity = LabelEncoder()
y_porosity = le_porosity.fit_transform(d5f2["hair_porosity"])

le_brk = LabelEncoder()
y_brk = le_brk.fit_transform(d5f2["hair_breakage"])

# Save encoder
with open("porosity.pkl", "wb") as f:
    pickle.dump(le_porosity, f)


# Save encoder
with open("breakage.pkl", "wb") as f:
    pickle.dump(le_brk, f)
    
print("✅ Saved  encoder for  porosity.pkl n breakage.pkl")


✅ Saved  encoder for  porosity.pkl n breakage.pkl


## test the models:

In [62]:
import pickle

# Load encoders
with open("porosity.pkl", "rb") as f:
    porosity_encoder = pickle.load(f)

with open("breakage.pkl", "rb") as f:
    breakage_encoder = pickle.load(f)

# Test with a new row
test_por = ["High"]
test_brk = ["None"]

print("Encoded porosity:", porosity_encoder.transform(test_por)[0])
print("Encoded breakage:", breakage_encoder.transform(test_brk)[0])



Encoded porosity: 0


ValueError: y contains previously unseen labels: 'None'

In [46]:
row_index = 100  # choose your row

row_porosity = d5f2.loc[row_index, "hair_porosity"]
row_breakage = d5f2.loc[row_index, "hair_breakage"]

print("Original:", row_porosity,":", row_breakage)

# Encode using loaded encoders
enc_por = porosity_encoder.transform([row_porosity])[0]
enc_brk = breakage_encoder.transform([row_breakage])[0]

print("Encoded:", enc_por,":", enc_brk)


Original: Medium : Medium Breakage
Encoded: 2 : 4


In [ ]:
extreme-low 1
low         3
medium      4
high        2
extreme-high 0